In [ ]:
import string
import numpy as np
import PIL.Image

from os import listdir
from pickle import dump, load

from numpy import array
from numpy import argmax
import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
#from keras.preprocessing.text import Tokenizerb
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.callbacks import ModelCheckpoint

In [ ]:
def extract_features(filename):
    model = VGG16()  
    print(model.summary())
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    print(model.summary())
    image = load_img(filename, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    return feature

def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        print(yhat)
        yhat = argmax(yhat)
        word = word_for_id(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

tokenizer = load(open('tokenizer.pkl', 'rb'))
max_length = 34

In [ ]:
model = load_model(r'F:\Venkatesh\PROJECTS\Image Caption Generator\Training\models\model_6.h5')
path = r'F:\Venkatesh\PROJECTS\Image Caption Generator\Flask\uploads\47871819_db55ac4699.jpg'
photo = extract_features(path)

description = generate_desc(model, tokenizer, photo, max_length)
print(description)


In [ ]:
!pip install "gTTS==2.1.1"

In [ ]:
import gTTs

In [ ]:
# Import the required module for text 
# to speech conversion
#import gTTS
#from gtts import *
#from gtts import gTTS 

from gtts import gTTS

# This module is imported so that we can 
# play the converted audio
import os
  
# The text that you want to convert to audio
mytext = description
  
# Language in which you want to convert
language = 'en'
  
# Passing the text and language to the engine, 
# here we have marked slow=False. Which tells 
# the module that the converted audio should 
# have a high speed
myobj = gTTS(text=mytext, lang=language, slow=False)
  
# Saving the converted audio in a mp3 file named
# welcome 
myobj.save("welcome.mp3")
  
# Playing the converted file
os.system("mpg321 welcome.mp3")

In [ ]:
from pickle import load
from numpy import argmax
import argparse
from keras.preprocessing.sequence import pad_sequences
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.models import load_model
import os
from flask import Flask , request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer


app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')
@app.route('/predict',methods = ['GET','POST'])
def upload():
    if request.method == 'POST':
        f = request.files['image']
        print("current path")
        basepath = os.path.dirname("__file__")
        print("current path", basepath)
        filepath = os.path.join(basepath,'uploads',f.filename)
        print("upload folder is ", filepath)
        f.save(filepath)
        text = modelpredict(filepath)
        return text
def extract_features(filename):
    print('features extracted')
    model = VGG16()
    #model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    image = load_img(filename, target_size=(224, 224))
    print('image loaded')
    image = img_to_array(image)
    print(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    print('model predicted')
    return feature
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
def generate_desc(model, tokenizer, photo, max_length):
    print("generate description")
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        print('sequence')
        yhat = model.predict([photo,sequence], verbose=0)
        print(yhat)
        yhat = argmax(yhat)
        word = word_for_id(yhat, tokenizer)
        print(word)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    print(in_text)
    return in_text
 
def modelpredict(filepath):
    tokenizer = load(open(r'F:\Venkatesh\PROJECTS\Image Caption Generator\tokenizer.pkl', 'rb'))
    max_length = 34
    model = load_model(r'F:\Venkatesh\PROJECTS\Image Caption Generator\model_18.h5')
    print('model loaded')
    photo = extract_features(filepath)
    description = generate_desc(model, tokenizer, photo, max_length)
    return description

if __name__ == '__main__':
    app.run(debug =False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Nov/2021 17:26:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2021 17:26:48] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2021 17:26:48] "GET /static/js/main.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2021 17:26:51] "GET /favicon.ico HTTP/1.1" 404 -


current path
current path 
upload folder is  uploads\109202756_b97fcdc62c.jpg
model loaded
features extracted
image loaded
[[[211. 224. 241.]
  [212. 225. 242.]
  [212. 225. 242.]
  ...
  [121. 121. 113.]
  [145. 144. 149.]
  [221. 227. 227.]]

 [[211. 224. 241.]
  [211. 224. 241.]
  [211. 224. 241.]
  ...
  [100. 101. 103.]
  [121. 122. 142.]
  [240. 239. 255.]]

 [[211. 224. 241.]
  [211. 224. 241.]
  [211. 224. 241.]
  ...
  [110. 116. 104.]
  [205. 206. 200.]
  [ 81.  71.  95.]]

 ...

 [[179. 160. 153.]
  [166. 158. 137.]
  [127. 114. 105.]
  ...
  [232. 236. 248.]
  [226. 230. 242.]
  [228. 232. 244.]]

 [[102.  83.  79.]
  [100.  85.  78.]
  [189. 167. 169.]
  ...
  [229. 233. 244.]
  [231. 235. 246.]
  [230. 234. 245.]]

 [[ 74.  61.  52.]
  [ 55.  47.  36.]
  [129. 114. 111.]
  ...
  [229. 233. 244.]
  [230. 234. 245.]
  [230. 234. 245.]]]
model predicted
generate description
sequence
[[8.8604670e-09 8.8005914e-09 1.8135086e-05 ... 8.6416057e-09
  2.9506666e-07 1.4755507e-07]]

127.0.0.1 - - [04/Nov/2021 17:28:00] "POST /predict HTTP/1.1" 200 -


[[3.1363030e-11 3.2993667e-11 4.0966210e-01 ... 3.0075244e-11
  8.9256064e-18 3.7041103e-16]]
endseq
startseq dog is running through the grass endseq


In [ ]:
#import sys
#!{sys.executable} -m pip install gevent

In [ ]:
#from flask import Flask